- Sky wirte test code at 2024-06-18.
- The aim is to test the Hikrobot industrial camera functionality.

In [9]:
import ctypes
import numpy as np
import cv2
import time
import platform
from MvCameraControl_class import *

# 0 设备扫描与开启

In [10]:
cam = MvCamera()
device_list = MV_CC_DEVICE_INFO_LIST()
n_ret = MvCamera.MV_CC_EnumDevices(MV_GIGE_DEVICE | MV_USB_DEVICE, device_list)
print(n_ret)
device_count = device_list.nDeviceNum
print(device_count)

0
1


In [11]:
device_info_ptr = device_list.pDeviceInfo[0]
device_info = device_info_ptr.contents

if device_info.nTLayerType == MV_USB_DEVICE:
    chUserDefinedName = bytearray()
    for i in range(256):
        if device_info.SpecialInfo.stUsb3VInfo.chUserDefinedName[i] == 0:
            break
        chUserDefinedName.append(device_info.SpecialInfo.stUsb3VInfo.chUserDefinedName[i])
    if len(chUserDefinedName) == 0:
        chUserDefinedName = b"Not defined name by users"
    print(f"  设备名: {chUserDefinedName.decode()}")

  设备名: Not defined name by users


In [12]:
n_ret = cam.MV_CC_CreateHandle(device_info)
if n_ret != 0:
    print(f"[ERROR] 创建设备句柄失败！错误码: {n_ret:#x}")

n_ret = cam.MV_CC_OpenDevice(MV_ACCESS_Exclusive, 0)
if n_ret != MV_OK:
    print(f"[ERROR] 打开设备失败！错误码: {n_ret:#x}")

In [ ]:
n_ret = cam.MV_CC_SetEnumValue("TriggerMode", MV_TRIGGER_MODE_OFF)
if n_ret != MV_OK:
    print(f"[WARNING] 设置触发模式失败: {n_ret:#x}")

try:
    # n_ret = cam.MV_CC_SetFloatValue("AcquisitionFrameRate", 30.0)
    n_ret = cam.MV_CC_SetBoolValue("AcquisitionFrameRateEnable", False)
    if n_ret == MV_OK:
        # print("  帧率设置为: 30.0 fps")
        print("  帧率自动控制已关闭")
except:
    pass

try:
    n_ret = cam.MV_CC_SetEnumValue("ExposureAuto", 2)  # 1 = 连续自动曝光
    cam.MV_CC_SetFloatValue("ExposureTimeUpper", 50000.0)
    if n_ret == MV_OK:
        print("  自动曝光已开启")
except:
    pass

try:
    # 将增益自动模式设置为连续（通常 0=Off, 1=Once, 2=Continuous）
    n_ret = cam.MV_CC_SetEnumValue("GainAuto", 2)
    cam.MV_CC_SetFloatValue("GainUpperLimit", 12.0)
    if n_ret == MV_OK:
        print("  自动增益已开启 (Continuous)")
    else:
        print(f"  [提示] 自动增益设置返回: {n_ret:#x}，上界为12.0")
except Exception as e:
    print(f"  设置自动增益时发生异常: {e}")

try:
    # Gamma值通常设为1.0（无校正）或2.2（sRGB标准，用于增强暗部细节）
    # 根据你的场景调整：物体颜色鲜艳且光照好可用1.0，光线较暗或需要提亮暗部可用2.2
    target_gamma = 1.0
    n_ret = cam.MV_CC_SetFloatValue("Gamma", target_gamma)
    if n_ret == MV_OK:
        print(f"  Gamma值已设置为: {target_gamma}")
    else:
        # 如果Gamma不支持，尝试设置GammaEnable
        n_ret = cam.MV_CC_SetBoolValue("GammaEnable", True)
        if n_ret == MV_OK:
            print("  Gamma校正已启用（使用默认曲线）")
except Exception as e:
    print(f"  设置Gamma时发生异常: {e}")

try:
    n_ret = cam.MV_CC_SetEnumValue("PixelFormat", PixelType_Gvsp_BayerRG8)
    if n_ret == MV_OK:
        print("  像素格式已设置为: Bayer RG 8")
    else:
        print(f"  [警告] 设置像素格式失败，错误码: {n_ret:#x}。将使用当前格式。")
except Exception as e:
    print(f"  设置像素格式时发生异常: {e}")

try:
    # 先尝试设置为自动白平衡（Continuous模式）
    n_ret = cam.MV_CC_SetEnumValue("BalanceWhiteAuto", 2)  # 2 = Continuous
    if n_ret == MV_OK:
        print("  自动白平衡已开启 (Continuous)")
    else:
        # 如果自动模式不支持，尝试设置手动白平衡系数（需根据场景调整）
        # 通常R和B通道需要调整，G通道保持1.0
        # cam.MV_CC_SetEnumValue("BalanceRatioSelector", 0)  # 选择Red通道
        # cam.MV_CC_SetFloatValue("BalanceRatio", 1.8)      # 设置Red系数
        # cam.MV_CC_SetEnumValue("BalanceRatioSelector", 2)  # 选择Blue通道
        # cam.MV_CC_SetFloatValue("BalanceRatio", 1.4)      # 设置Blue系数
        print("  自动白平衡不被支持，将使用默认或当前白平衡设置")
except Exception as e:
    print(f"  设置白平衡时发生异常: {e}")

try:
    # 启用锐化，提升边缘清晰度（有利于轮廓提取）
    n_ret = cam.MV_CC_SetBoolValue("SharpnessEnable", True)
    if n_ret == MV_OK:
        print("  锐化已启用")
except:
    pass

print("[INFO] 相机参数设置完成")

  帧率设置为: 30.0 fps
  自动曝光已开启
  自动增益已开启 (Continuous)
  Gamma值已设置为: 1.0
  像素格式已设置为: Bayer RG 8
  自动白平衡已开启 (Continuous)
[INFO] 相机参数设置完成


# 1. 取流

In [14]:
n_ret = cam.MV_CC_StartGrabbing()
if n_ret != MV_OK:
    print(f"[ERROR] 开始采集失败！错误码: {n_ret:#x}")

## 1.1 单帧

In [ ]:
# 帧输出信息结构体
stFrameInfo = MV_FRAME_OUT_INFO_EX()
# 清空结构体
memset(ctypes.byref(stFrameInfo), 0, ctypes.sizeof(stFrameInfo))
# 创建缓冲区 (按最大可能尺寸：3072*2048*2， 应对12位非Packed的2字节/像素情况)
data_buf_size = 3072 * 2048 * 2
pData = (ctypes.c_ubyte * data_buf_size)()

# 主动获取一帧图像 (超时时间1000ms)
nRet = cam.MV_CC_GetOneFrameTimeout(pData, data_buf_size, stFrameInfo, 1000)
if nRet != MV_OK:
    print(f"获取图像失败！错误码: {nRet:#x}")
else:
    print(f"获取图像成功！宽：{stFrameInfo.nWidth}, 高：{stFrameInfo.nHeight}, 像素格式枚举值：{stFrameInfo.enPixelType:#x}")
    
    # 将原始数据转换为 numpy 数组
    image_array = np.frombuffer(pData, dtype=np.uint8, count=stFrameInfo.nFrameLen)
    
    output_image = None # 用于存储最终转换后的BGR图像
    # Bayer RG 8
    if stFrameInfo.enPixelType == PixelType_Gvsp_BayerRG8:
        print("  处理: Bayer RG 8 格式")
        # 重塑为二维灰度图（每个像素点是一个Bayer颜色分量）
        bayer_2d = image_array.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
        output_image = cv2.cvtColor(bayer_2d, cv2.COLOR_BayerRG2RGB)
    
    else:
        print(f"[ERROR] 无法处理的未知像素格式: {stFrameInfo.enPixelType:#x}")

    if output_image is not None:
        print(f"[SUCCESS] 图像转换成功，尺寸: {output_image.shape}")
        hsv_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2HSV)
        
        # 定义红色的HSV阈值范围（红色在HSV色环上位于0°和180°附近，所以需要两个范围）
        # 注意：这些阈值需要根据您的实际红色框和光照环境进行微调！
        # H: 色相 (0-180), S: 饱和度 (0-255), V: 明度 (0-255)
        lower_red1 = np.array([0, 100, 100])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160, 100, 100])
        upper_red2 = np.array([180, 255, 255])
        
        # 根据阈值创建掩膜
        mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
        mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
        red_mask = cv2.bitwise_or(mask_red1, mask_red2)
        
        # 可选：进行形态学操作（如开运算、闭运算）去除噪声，连接断裂区域
        kernel = np.ones((5,5), np.uint8)
        red_mask_cleaned = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
        red_mask_cleaned = cv2.morphologyEx(red_mask_cleaned, cv2.MORPH_OPEN, kernel)
        
        # 在原始图像上显示掩膜，用于调试
        cv2.imwrite('debug_red_mask.jpg', red_mask_cleaned)
        
        # 在掩膜中查找轮廓
        contours, _ = cv2.findContours(red_mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # 假设最大的红色轮廓是我们的目标框
            largest_contour = max(contours, key=cv2.contourArea)
            
            # 计算轮廓的像素面积和外接矩形
            pixel_area = cv2.contourArea(largest_contour)
            x, y, w, h = cv2.boundingRect(largest_contour)
            
            # 在原始图像上绘制轮廓和外接矩形（用于可视化验证）
            output_with_contour = output_image.copy()
            cv2.drawContours(output_with_contour, [largest_contour], -1, (0, 255, 0), 3) # 绿色轮廓
            cv2.rectangle(output_with_contour, (x, y), (x+w, y+h), (255, 0, 0), 2) # 蓝色外接框
            cv2.imwrite('output_with_detection.jpg', output_with_contour)
            
            # 计算轮廓的中心点像素坐标
            center_x_pixel = x + w // 2
            center_y_pixel = y + h // 2
            cv2.circle(output_with_contour, (center_x_pixel, center_y_pixel), 10, (0, 0, 255), -1) # 红色中心点
            cv2.imwrite('output_with_center.jpg', output_with_contour)
            
            print(f"[检测成功] 红色轮廓像素中心: ({center_x_pixel}, {center_y_pixel})")
            print(f"          外接矩形宽高: {w} x {h} 像素, 轮廓面积: {pixel_area:.1f} 像素")
            
            # ========== 第二步：与已知轮廓对比，计算三维信息 ==========
            # 已知条件：您提到的目标物理尺寸（请根据实际情况修改）
            # 情况A: 如果红色框是边长为50cm的正方形
            # KNOWN_WIDTH = 0.5   # 单位：米 (m)
            # KNOWN_HEIGHT = 0.5  # 单位：米 (m)
            # 情况B: 如果红色框是直径为50cm的圆形（请用此选项）
            KNOWN_DIAMETER = 0.02  # 单位：米 (m)
            
            # 【关键】您需要提供相机的内参矩阵（通过相机标定获得）
            # 这是一个3x3矩阵，格式为: 
            # fx, 0,  cx
            # 0,  fy, cy
            # 0,  0,  1
            # 其中 (fx, fy) 是以像素为单位的焦距，(cx, cy) 是图像中心（主点）
            # 以下是示例值，您必须替换为通过标定得到的真实数据！
            # 假设您的相机分辨率是3072x2048，这里是一个估算值，精度不足以用于实际抓取
            camera_matrix = np.array([
                [2500.0, 0,     1536.0],  # fx, 0,  cx
                [0,     2500.0, 1024.0],  # 0,  fy, cy
                [0,     0,     1.0]
            ])
            print(f"[提醒] 当前使用示例相机内参，请务必进行相机标定获取准确参数！")
            
            # 方法1：利用外接矩形宽度（或高度）与已知物理尺寸的比例估算深度（Z）
            # 这是基于小孔成像模型的相似三角形原理: Z = (已知物理宽度 * 焦距) / 像素宽度
            # 使用 fx 对应宽度，fy 对应高度
            focal_length_x = camera_matrix[0, 0]
            # estimated_distance_z = (KNOWN_WIDTH * focal_length_x) / w
            # print(f"[三维估算] 方法1 - 基于宽度:")
            # print(f"          估算距离 (Z): {estimated_distance_z:.3f} 米")
            
            # 方法2：如果目标是圆形，且已知其物理直径，可以使用轮廓面积进行更鲁棒的估算
            # 假设轮廓近似圆形: 像素半径 r_pixel ≈ sqrt(像素面积 / π)
            # 则 Z = (已知物理直径 * 焦距) / (2 * r_pixel)
            # 以下是示例（如果您使用圆形）：
            r_pixel = np.sqrt(pixel_area / np.pi)
            estimated_distance_z = (KNOWN_DIAMETER * focal_length_x) / (2 * r_pixel)
            print(f"[三维估算] 方法2 - 基于圆形面积:")
            print(f"          估算距离 (Z): {estimated_distance_z:.3f} 米")
            
            # 将像素中心坐标转换到相机三维坐标系 (X, Y, Z)
            # 公式: 
            # X = (中心点x像素 - cx) * Z / fx
            # Y = (中心点y像素 - cy) * Z / fy
            cx = camera_matrix[0, 2]
            cy = camera_matrix[1, 2]
            fx = camera_matrix[0, 0]
            fy = camera_matrix[1, 1]
            
            estimated_x = (center_x_pixel - cx) * estimated_distance_z / fx
            estimated_y = (center_y_pixel - cy) * estimated_distance_z / fy
            
            print(f"[三维坐标] 红色框在相机坐标系下的位置:")
            print(f"          X (水平): {estimated_x:.3f} 米")
            print(f"          Y (垂直): {estimated_y:.3f} 米")
            print(f"          Z (深度): {estimated_distance_z:.3f} 米")
            
            # ========== 第三步：坐标转换到机械臂基座坐标系 ==========
            # 这是通过“手眼标定”得到的变换矩阵完成的
            # 您需要提供一个4x4的齐次变换矩阵 (旋转矩阵R 3x3 + 平移向量t 3x1)
            # 假设您已经完成了手眼标定，以下是示例矩阵（单位矩阵表示无变换，仅作演示）
            # 您必须替换为真实的变换矩阵！
            hand_eye_matrix = np.eye(4) 
            print(f"[提醒] 当前使用单位矩阵作为手眼变换矩阵，请务必进行手眼标定获取准确矩阵！")
            
            # 构造相机坐标系下的点 (X, Y, Z, 1)
            point_in_camera = np.array([estimated_x, estimated_y, estimated_distance_z, 1.0])
            
            # 转换到机械臂基座坐标系
            point_in_robot_base = hand_eye_matrix @ point_in_camera.T
            
            print(f"[机械臂坐标] 红色框在机械臂基座坐标系下的位置:")
            print(f"          X: {point_in_robot_base[0]:.3f} 米")
            print(f"          Y: {point_in_robot_base[1]:.3f} 米")
            print(f"          Z: {point_in_robot_base[2]:.3f} 米")
            
        else:
            print("[检测失败] 未找到红色轮廓，请检查HSV阈值或光照条件。")
        # cv2.imshow('Converted Image', output_image)
        # cv2.waitKey(1)
        cv2.imwrite('first_frame_debug.jpg', output_image)
    else:
        print("[ERROR] 图像转换失败，无法得到输出图像。")

获取图像成功！宽：3072, 高：2048, 像素格式枚举值：0x1080009
  处理: Bayer RG 8 格式
[SUCCESS] 图像转换成功，尺寸: (2048, 3072, 3)
[检测失败] 未找到红色轮廓，请检查HSV阈值或光照条件。


## 1.2 连续取流

In [23]:
# 创建帧信息结构体
stFrameInfo = MV_FRAME_OUT_INFO_EX()
memset(ctypes.byref(stFrameInfo), 0, ctypes.sizeof(stFrameInfo))

# 创建缓冲区
data_buf_size = 3072 * 2048 * 2
pData = (ctypes.c_ubyte * data_buf_size)()

while True:
    nRet = cam.MV_CC_GetOneFrameTimeout(
        pData,
        data_buf_size,
        stFrameInfo,
        1000
    )

    if nRet != MV_OK:
        print(f"[WARN] 获取图像失败: {nRet:#x}")
        continue

    # numpy
    image_array = np.frombuffer(
        pData,
        dtype=np.uint8,
        count=stFrameInfo.nFrameLen
    )

    output_image = None

    # Bayer RG8
    if stFrameInfo.enPixelType == PixelType_Gvsp_BayerRG8:
        bayer_2d = image_array.reshape(
            (stFrameInfo.nHeight, stFrameInfo.nWidth)
        )
        output_image = cv2.cvtColor(
            bayer_2d,
            cv2.COLOR_BAYER_RG2RGB

        )
    else:
        print(f"[ERROR] 不支持的像素格式: {stFrameInfo.enPixelType:#x}")
        continue

    # ================== 红色检测 ==================
    hsv_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(
        red_mask,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)

        cv2.rectangle(
            output_image,
            (x, y),
            (x + w, y + h),
            (0, 255, 0),
            2
        )

        cx = x + w // 2
        cy = y + h // 2
        cv2.circle(output_image, (cx, cy), 6, (0, 0, 255), -1)

    # ================== 显示 ==================
    cv2.imshow("Camera Stream", output_image)
    cv2.imshow("Red Mask", red_mask)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:  # q 或 ESC
        break

In [24]:
cv2.destroyAllWindows()

# 2. 结束处理

In [8]:
n_ret = cam.MV_CC_StopGrabbing()
if n_ret != MV_OK:
    print(f"[ERROR] 结束采集失败！错误码: {n_ret:#x}")

n_ret = cam.MV_CC_CloseDevice()
if n_ret != MV_OK:
    print(f"[ERROR] 关闭设备失败！错误码: {n_ret:#x}")
    
n_ret = cam.MV_CC_DestroyHandle()
if n_ret != MV_OK:
    print(f"[ERROR] 摧毁句柄失败！错误码: {n_ret:#x}")